# N-Layer Neural Network | Text Colour Predictor

Task:
- Build a scalable feed-forward neural network.
- Input values of RGB 'background colour'.
- Predict if light or dark coloured text should be used over the RGB colour to make the text readable.

Task mapping:
- Objects of interest: RGB vectors (3 $\times$ 1 dimension).
- Labels: light text versus dark text.

Resources:
- [Feed-forward NN playground](https://playground.tensorflow.org)
- [Activation functions](https://missinglink.ai/guides/neural-network-concepts/7-types-neural-network-activation-functions-right/)

## Import libraries:

In [ ]:
# Import libraries:
import numpy as np # For linear algebra.
import pandas as pd # For data processing, CSV file I/O (e.g. pd.read_csv).
import matplotlib.pyplot as plt # For data visualisation.
from PIL import Image, ImageDraw, ImageFont, ImageEnhance # For data visualisation.
%matplotlib inline

## RGB class and tool functions:

In [ ]:
class Error(Exception):
    """Base class for exceptions in this module."""
    pass

class InputError(Error):
    '''Exception raised for errors in the input.

    Attributes:
        expr -- Input expression in which the error occurred.
        msg  -- Explanation of the error.'''
    def __init__(self, expr, msg):
        self.expr = expr
        self.msg = msg

class RGB():
    '''Defined with values for RGB as input.
    
    Attributes:
        RGB -- Input RGB values should range from 0 to 255.
        hex -- Automatically converts RGB to hex values.'''
    def __init__(self, R, G, B):
        for X in [R, G, B]:
            if (X < 0) or (X > 255):
                raise InputError(X, 'Not an RGB value.')
        self.R = R
        self.G = G
        self.B = B
        self.RGB = (R, G, B)
        self.hex = '#{:02X}{:02X}{:02X}'.format(self.R,self.G,self.B)

def generate_RGB_data(X, extreme=False, extreme_magnitude=200):
    '''Generates a list filled with X number of RGB class values.
    Optional: generate cols that are v. dark + v. light for training.
    
    Attributes:
        X -- Number of desired RGB instances.
        extreme -- Boolean to generate v. dark + v. light cols.
        extreme_magnitude -- Int between 1 and 254.'''
    if extreme == True:
        cols = []
        for x in range(X):
            minimum = extreme_magnitude*(x%2)
            maximum = 255-(extreme_magnitude*(not x%2))
            rgb = RGB(np.random.randint(low=minimum, high=maximum),
                      np.random.randint(low=minimum, high=maximum),
                      np.random.randint(low=minimum, high=maximum))
            cols.append(rgb)
        return cols
                        
    else:
        return [RGB(np.random.randint(0, 255),
                    np.random.randint(0, 255),
                    np.random.randint(0, 255))
                for i in range(X)]

def display_RGB_colour(colour, font_col='#000'):
    '''Will draw a box of given colour;
    and fill with text of given font colour.
    
    Attributes:
        colour -- String containing a RGB or hex value.
        font_col -- String containing a RGB or hex value.'''
    img = Image.new(mode='RGB', size=(100, 100), color=colour)
    img_draw = ImageDraw.Draw(img)
    img_draw.text((36, 45), 'Text', fill=font_col)
    plt.imshow(img)
    plt.show();

## Data visualisation:

In [ ]:
# Test the RGB class and data visualisation tool functions:
colours = generate_RGB_data(X=1, extreme=True, extreme_magnitude=200)

for colour in colours:
    print('RGB:', colour.RGB, 'Hex:', colour.hex)
    display_RGB_colour(colour=colour.RGB, font_col='#fff')

## Generate data:

NB: Change "extreme_magnitude" to adjust the level of noise. Default has no noise (i.e. easy to model).

In [ ]:
np.random.seed(42) # Optional: set seed for data generation.
extreme_magnitude = 100 # Optional: set magnitude lower for higher error.

data = pd.DataFrame([x.RGB for x in generate_RGB_data(X=500,
                                                      extreme=True,
                                                      extreme_magnitude=extreme_magnitude)],
                     columns=['R', 'G', 'B'])

display('Training set:', data)

## Assign Labels

NB: This the 'lazy' method.

In [ ]:
# Import libraries:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
clusterer = AgglomerativeClustering(n_clusters=2, linkage='ward').fit(data.values)
y = clusterer.labels_

In [ ]:
for i, label in enumerate(y[:2]):
    
    if label == 1: # NB: must check for most appriate label-to-class assignment.
        print('---> light text')
        display_RGB_colour(colour=tuple(data.iloc[i, :]), font_col='#fff')
        
    else:
        print('---> dark text')
        display_RGB_colour(colour=tuple(data.iloc[i, :]), font_col='#000')

## Train/test split:

In [ ]:
# Import libraries:
from sklearn.model_selection import train_test_split

In [ ]:
# Split data into training & testing sets:
train_temp, test_temp = train_test_split(data.join(pd.Series(y, name='y')))

display(train_temp.head())
display(test_temp.head())

## Preprocessing:

In [ ]:
# Import libraries:
from sklearn.preprocessing import normalize

train = pd.DataFrame(np.insert(normalize(X=train_temp.values[:, :3]), 3, train_temp.values[:, 3], axis=1),
                     index=train_temp.index,
                     columns=train_temp.columns)

test = pd.DataFrame(np.insert(normalize(X=test_temp.values[:, :3]), 3, test_temp.values[:, 3], axis=1),
                    index=test_temp.index,
                    columns=test_temp.columns)

display(train.head())
display(test.head())

## Building the NN:

In [ ]:
# Define Neuron class:
class NeuralNetwork():
    def __init__(self, X, y, bias=1, eta=0.1, n_nodes=2, n_layers=2, Ws=None, linear=False):
        '''Initialise internal state of network. CAUTION: when setting own Ws param,
        make sure the matrix dimensions are correct.
        
        Attributes:
        X -- Initial input vector; should be a numpy array or matrix.
        y -- Initial y_true; should be numpy array.
        Ws -- Optional. If given should be a list of numpy arrays.'''
        # Create list of LAYERS:
        self.layers = []
        self.layers.append(X) # Append input layer.
        for i in range(n_layers-1):
            self.layers.append(np.zeros((n_nodes, 1))) # Append hidden layers.
        self.layers.append(np.zeros(y.shape)) # Append output layer.

        # Create list of WEIGHTS:
        if Ws is None:
            self.Ws = []
            for i in range(n_layers):
                self.Ws.append(np.random.rand(self.layers[i+1].shape[0], self.layers[i].shape[0]))
        else:
            self.Ws = Ws

        # Create list of BIASES:
        self.biases = []
        for i in range(n_layers):
            self.biases.append(np.ones((self.Ws[i].shape[0], self.layers[i].shape[1]))*bias) # Multiply bias.

        # Set the other parameters:
        self.y_true = y
        self.eta = eta
        self.linear = linear
        self.n_layers = n_layers
    
    def activ_func(self, x):
        '''Activation function used during forward pass.'''
        # For linear:
        if self.linear is True:
            return x
        # For sigmoid:
        else:
            return 1.0/(1.0 + np.exp(-x))
    
    def forwardpass(self):
        '''Runs the forward pass algorithm using the internal state (via self).'''
        for i in range(self.n_layers):
            self.layers[i+1] = self.activ_func(np.dot(self.Ws[i], self.layers[i]) + self.biases[i])
        
    def activ_deriv(self, x):
        '''Derivative of the activation function used during backpropagation.'''
        # For linear:
        if self.linear is True:
            return 1
        # For sigmoid:
        else:
            return self.activ_func(x)*(1-self.activ_func(x))
    
    def error_deriv(self):
        '''Derivative of the error function used during backpropagation.'''
        return -(self.y_true-self.layers[-1])
    
    def error(self):
        '''Error function.'''
        return ((self.y_true-self.layers[-1])**2)*0.5
    
    def backprop(self):
        '''Runs backpropagation algorithm using the internal state (via self):
        (1) applies chain rule to find derivative of loss function;
        (2) updates the weights and biases with the gradient of the loss function.'''
        # Initialise lists to contain deltas:
        deltas = []
        
        # Iterate over n number of layers and calculate delta:
        for i in reversed(range(self.n_layers)): # NB: reversed for backpropagation.
            # Calculate the deriv wrt. activation:
            d_activ = self.activ_deriv(x=np.dot(self.Ws[i], self.layers[i]))
            
            # Delta for output layer:
            if i == self.n_layers-1:
                delta = self.error_deriv() * d_activ
                
            # Delta for subsequent layers:
            else:
                delta = np.dot(deltas[0].T, self.Ws[i+1]).T * d_activ # NB: uses the prev delta and prev layer.
                
            # Save delta to list:
            deltas.insert(0, delta) # NB: undo reversed order.

        # Iterate over deltas and apply both kinds of updates:
        for i in range(self.n_layers):
            # Update weight:
            self.Ws[i] += -self.eta * np.dot(deltas[i], self.layers[i].T)
            
            # Update bias:
            self.biases[i] += -self.eta * deltas[i] * self.biases[i]

    def fit(self, Xs, ys, iterations=1):
        '''Applies the forward pass and backpropagation algorithms in sequence to fit given training data.
        
        Attributes:
        iterations -- Number of times to repeat the sequence over whole dataset, aka epochs.'''
        y_preds = []
        
        for iteration in range(iterations): # Per iteration.
            for i, X in enumerate(Xs): # Per data point.
                # Reset inputs:
                self.layers[0] = X  # X assigned to input layer.
                self.y_true = ys[i] # y assigned to y_true.
                
                self.forwardpass()
                self.backprop()
                
                # Save the final interation of output layer:
                if iteration == iterations-1:
                    y_preds.append(self.layers[-1])
                    
        return np.array(y_preds)
    
    def predict(self, Xs):
        '''Applies forward pass using the internal state to the given input data (Xs).
        
        Attributes:
        Xs -- Input data.'''
        y_preds = []
        
        for X in Xs: # Per data point.
            self.layers[0] = X # X assigned to input layer.
            self.forwardpass()
            y_preds.append(self.layers[-1])
            
        return np.array(y_preds)
    
    def display_test_results(self, Xs, y_preds):
        '''Will plot a figure of a given colour (via Xs) and its predicted text colour (via y_preds).
        NB: specific to the "text predictor" scenario.
        
        Attributes:
        Xs -- Input data.
        y_preds -- Predicted colours.'''
        for i, y in enumerate(y_preds):
            if y == 0:
                print('\n--->\t{}:\tlight text'.format(y))
                display_RGB_colour(colour=tuple(Xs[i, :]), font_col='#fff')

            else:
                print('\n--->\t{}:\dark text'.format(y))
                display_RGB_colour(colour=tuple(Xs[i, :]), font_col='#000')

## Use NN on a dummy example:

NB: the idea behind using the dummy example is tha tit is easy to calculate by hand.

The results should be as follows:
- First ***forward pass*** output: $\begin{bmatrix} 2 \\ 2 \end{bmatrix}$
- ***Backpropagation***...
    - ... hidden layer update: $\begin{bmatrix} -1 & 0.1 \\ 0 & 0.8 \end{bmatrix}$
    - ... output layer update: $\begin{bmatrix} 0.9 & -0.2 \\ -1.2 & 0.6 \end{bmatrix}$
- Second ***forward pass*** output: $\begin{bmatrix} 1.66 \\ 0.32 \end{bmatrix}$

In [ ]:
# Setup inputs:
X = np.array([0, 1]).reshape((2,1))
y = np.array([1, 0]).reshape((2,1))
Ws = [np.array([[-1, 0], [0, 1]], dtype=float),
      np.array([[1, 0], [-1, 1]], dtype=float)]

# Initialise NN:
NN = NeuralNetwork(X=X,
                   y=y,
                   bias=1,
                   eta=0.1,
                   n_nodes=2,
                   n_layers=2,
                   Ws=Ws,
                   linear=True)

# Use NN:
NN.forwardpass()
print('\nForward Pass:\noutput:\n{}'.format(NN.layers[-1]))

NN.backprop()
print('\nBackpropagation:\nhidden:\n{}\noutput:\n{}'.format(NN.Ws[-2], NN.Ws[-1]))

NN.forwardpass()
print('\nForward Pass:\noutput:\n{}'.format(NN.layers[-1]))

## Train NN on the training set:

In [ ]:
# Setup inputs:
input_X = train.values[0, :3].reshape((3,1))
input_y = train.values[0, 3].reshape((1,1))
print('X:\n{}\ny:\n{}'.format(input_X, input_y))

# Initialise NN:
NN = NeuralNetwork(X=input_X,
                   y=input_y,
                   bias=1,
                   eta=0.1,
                   n_nodes=5,
                   n_layers=3,
                   Ws=None,
                   linear=False)

# Use NN:
NN.forwardpass()
print('\nForward Pass:\noutput:\n{}'.format(NN.layers[-1]))

NN.backprop()
print('\n...')

NN.forwardpass()
print('\nForward Pass:\noutput:\n{}'.format(NN.layers[-1]))

In [ ]:
%%time

# Setup inputs:
tr_i = train.shape[0]
train_Xs = train.values[:, :3].reshape((tr_i, 3, 1))
train_ys = train.values[:,  3].reshape((tr_i, 1, 1))
print('Shapes of inputs:', train_Xs.shape, train_ys.shape)

# Train NN:
train_y_preds = NN.fit(Xs=train_Xs, ys=train_ys, iterations=1000)
print('Shape of y_preds:', train_y_preds.shape)

In [ ]:
# Check training predictions:
print(train_y_preds.reshape(tr_i))

In [ ]:
# Check NN layers:
for i, layer in enumerate(NN.layers):
    print('Layer #{}.\n{}\n'.format(i, layer))

In [ ]:
# Training results:
# NB: the y predictions are rounded!
train_results = pd.DataFrame({'y_true': train.y.values,
                              'y_pred': np.round(train_y_preds).reshape((tr_i,)).astype(int),
                              'same': train.y.values == np.round(train_y_preds).reshape((tr_i,)).astype(int)})

display(train_results.loc[train_results.y_pred==1])
display(train_results.loc[train_results.y_pred==0])

print('{}% error'.format(round(len(train_results[train_results.same==False]) / len(train_results) * 100)))

## Test NN on the training set:

In [ ]:
# Setup inputs:
te_i = test.shape[0]
test_Xs = test.values[:, :3].reshape((te_i, 3, 1))
test_ys = test.values[:,  3].reshape((te_i, 1, 1))
print('Shapes of inputs:', test_Xs.shape, test_ys.shape)

# Test NN:
test_y_preds = NN.predict(Xs=test_Xs)
print('Shape of y_preds:', test_y_preds.shape)

In [ ]:
# Check testing predictions:
print(test_y_preds.reshape(test_y_preds.shape[0]))

In [ ]:
# Testing results:
# NB: the y predictions are rounded!
test_results = pd.DataFrame({'y_true': test.y.values,
                             'y_pred': np.round(test_y_preds).reshape((te_i,)).astype(int),
                             'same': test.y.values == np.round(test_y_preds).reshape((te_i,)).astype(int)})

display(test_results.head(15))

print('{}% error'.format(round(len(test_results[test_results.same==False]) / len(test_results) * 100)))

# Compare against NN via scikit-learn

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
# Setup the sklearn neural network
sklearn_NN = MLPClassifier(activation='logistic',
                           solver='sgd',
                           max_iter=1000) # NB: 'logistic' and stochastic gd chosen for fair comparison.

# View description
sklearn_NN

In [ ]:
# Optimise parameters via a gridsearch:
sklearn_grid = GridSearchCV(estimator=sklearn_NN,
                            param_grid={'hidden_layer_sizes': [5, 50, 100],
                                        'learning_rate_init': [0.1, 0.01, 0.001],
                                        'alpha': [0.1, 0.01, 0.001]},
                            cv=5)

In [ ]:
# Train sklearn NN:
sklearn_grid.fit(X=train.iloc[:, :3].values,
                 y=train.y.values)

print('best parameters:\t{}'.format(sklearn_grid.best_params_))

In [ ]:
# Test sklearn NN:
sklearn_y_preds = sklearn_grid.predict(X=test.iloc[:, :3].values)

In [ ]:
# Display the sklearn predictions:
sklearn_results = pd.DataFrame({'y_true': test.y.values,
                                'y_pred': sklearn_y_preds,
                                'same': test.y.values == sklearn_y_preds})

display(sklearn_results.head(10))

# Print error:
print('{}% error'.format(round(len(sklearn_results[sklearn_results.same==False]) / len(sklearn_results) * 100)))

In [ ]:
# The end.